In [4]:
aois = ["Bab el-Mandeb Strait", "Cape of Good Hope", "Suez Canal"]
countries_of_interest = [
    "Egypt",
    "Yemen",
    "Djibouti",
    "Eritrea",
    "Saudi Arabia",
    "Jordan",
]
ISO_COUNTRIES = [818, 887, 262, 232, 682, 400]
START_DATE = "2023-01-01"

In [46]:
import logging

import os
import sys
from os.path import join

import pandas as pd

import git

git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(join(git_root, "src", "red-sea-monitoring"))
from acled import *
from visuals import *

from datetime import date

logger = logging.getLogger()
logging.basicConfig(format="%(asctime)s %(message)s", level=logging.INFO)

from plotnine import *

%reload_ext autoreload
%autoreload 2

# Conflict Location and Trends Monitor

This section examines how the conflict events in the countries of the red sea region have progressed since the crisis started in October 7th.

## Data

The Armed Conflict Location & Event Data Project (ACLED) is a disaggregated data collection, analysis, and crisis mapping project. ACLED collects information on the dates, actors, locations, fatalities, and types of all reported political violence and protest events around the world. The raw data is available through a license obtained by the World Bank. 

## Insights

To match the conflict analysis with the maritime trade trends as [previously seen on this web-book](https://datapartnership.org/red-sea-monitoring/notebooks/ports/README.html), we aggregated the ACLED data to show weekly trends near the ports of interest. 

### Visualizing conflict events between January 1st 2023 and February 27th 2024

In [6]:
data = acled_api(
    email_address=os.environ.get("ACLED_EMAIL"),
    access_key=os.environ.get("ACLED_KEY"),
    country=ISO_COUNTRIES,
    start_date="2023-07-08",
    end_date=date.today().isoformat(),
)

data2 = acled_api(
    email_address=os.environ.get("ACLED_EMAIL"),
    access_key=os.environ.get("ACLED_KEY"),
    country=ISO_COUNTRIES,
    start_date="2023-01-01",
    end_date="2023-07-07",
)

data = pd.concat([data, data2])

In [7]:
data["latitude"] = data["latitude"].astype("float64")
data["longitude"] = data["longitude"].astype("float64")
data["fatalities"] = data["fatalities"].astype("int")

In [8]:
import geopandas as gpd
from shapely import Point


def convert_to_gdf(data):
    geometry = [Point(xy) for xy in zip(data["longitude"], data["latitude"])]
    gdf = gpd.GeoDataFrame(data, geometry=geometry, crs="EPSG:4326")

    return gdf

In [9]:
grouped_data = convert_to_gdf(
    data.groupby(["latitude", "longitude"])["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
grouped_data.rename(
    columns={"sum": "nr_fatalities", "count": "nr_events"}, inplace=True
)

In [10]:
m = grouped_data.explore(column="nr_events", zoom_start=5.1, marker_kwds={"radius": 4})
m

### Ports of Interest in the Red Sea Region

In [11]:
ports_red_sea = gpd.read_file(
    join(git_root, "data", "red_sea_ports.geojson"), driver="GeoJSON"
)

In [12]:
import folium

radius = 50 * 1000
ports_red_sea = ports_red_sea.to_crs(epsg=32633)
ports_red_sea["geometry"] = ports_red_sea["geometry"].apply(lambda x: x.buffer(radius))


folium.GeoJson(
    ports_red_sea,
    name="Ports",
    # Customize the style if needed
    style_function=lambda feature: {
        "color": "#CC5500",
        "weight": 2,
        "fillColor": "#CC5500",
    },
).add_to(m)

m

c:\Users\sahit\anaconda3\envs\conflict-analyses\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.


In [13]:
data["event_date"] = pd.to_datetime(data["event_date"])

gdf = convert_to_gdf(data)

In [14]:
conflict_by_country = (
    data.groupby(["country", pd.Grouper(key="event_date", freq="W")])["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
conflict_by_country.rename(
    columns={"sum": "nrFatalities", "count": "nrEvents"}, inplace=True
)

In [15]:
conflict_by_country_event = (
    data.groupby(
        [
            "country",
            "event_type",
            "sub_event_type",
            pd.Grouper(key="event_date", freq="W"),
        ]
    )["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
conflict_by_country_event.rename(
    columns={"sum": "nrFatalities", "count": "nrEvents"}, inplace=True
)

In [16]:
conflict_by_port = gpd.sjoin_nearest(
    ports_red_sea.to_crs(epsg=32633),
    gdf.to_crs(epsg=32633),
    max_distance=150 * 1000,
    distance_col="distance",
    how="right",
)

In [17]:
conflict_by_port = conflict_by_port[
    conflict_by_port["country_left"] == conflict_by_port["country_right"]
]
conflict_by_port = conflict_by_port[~(conflict_by_port["index_left"].isnull())]

In [18]:
conflict_by_port = conflict_by_port[
    conflict_by_port["country_left"].isin(["Yemen", "Jordan"])
]

In [19]:
conflict_by_port = (
    conflict_by_port.groupby(
        [
            "portid",
            "portname",
            "fullname",
            "event_type",
            pd.Grouper(key="event_date", freq="W"),
            "country_right",
        ]
    )["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
conflict_by_port.rename(
    columns={"sum": "nrFatalities", "count": "nrEvents"}, inplace=True
)

In [20]:
conflict_event_coverage = round(
    100
    * conflict_by_port["nrEvents"].sum()
    / conflict_by_country[conflict_by_country["country"].isin(["Yemen", "Jordan"])][
        "nrEvents"
    ].sum(),
    2,
)
conflict_event_coverage

66.73

In [23]:
from datetime import datetime
start_reference_date = "2019-01-01"
conflict_date = datetime(2023, 10, 7)
crisis_date = datetime(2023, 11, 17)

In [45]:
from bokeh.plotting import figure, show, output_notebook
import bokeh
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs

output_notebook()

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

conflict_by_country = conflict_by_country.sort_values(by="country", ascending=False)

tabs = []
measure_names = {
    "nr_events": "Number of Conflict Events",
    "fatalities": "Number of Fatalities",
}
measure_colors = {"nrEvents": "#4E79A7", "fatalities": "#F28E2B"}
# acled_adm0 = get_acled_by_admin(syria_adm2_crs, acled, columns = ['ADM2_EN', 'ADM1_EN'])
for country in list(conflict_by_country["country"].unique()):
    tabs.append(
        Panel(
            child=get_bar_chart(
                conflict_by_country,
                f"Weekly Conflict Trend in {country}",
                "Source: ACLED",
                subtitle="",
                category="country",
                measure="nrEvents",
                color_code=measure_colors["nrEvents"],
                category_value=country,
                crisis_date=crisis_date,
                conflict_date=conflict_date,
            ),
            title=country.title(),
        )
    )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

In [26]:
port_names = (
    conflict_by_port[["portname", "fullname"]]
    .set_index("portname")
    .to_dict()["fullname"]
)

In [27]:
conflict_by_port.sort_values(by=["country_right", "event_date"], inplace=True)

In [28]:
event_types = list(conflict_by_port["event_type"].unique())

In [47]:
output_notebook()
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

conflict_by_port = conflict_by_port.sort_values(by="portname", ascending=True)

tabs = []
measure_names = {
    "nr_events": "Number of Conflict Events",
    "fatalities": "Number of Fatalities",
}
measure_colors = {"nrEvents": "#4E79A7", "fatalities": "#F28E2B"}
# acled_adm0 = get_acled_by_admin(syria_adm2_crs, acled, columns = ['ADM2_EN', 'ADM1_EN'])
for port in list(conflict_by_port["portname"].unique()):
    tabs.append(
        Panel(
            child=get_stacked_bar_chart(
                conflict_by_port[conflict_by_port["portname"] == port],
                f"Weekly Conflict Trend in {port_names[port]}",
                "Source: ACLED",
                date_column="event_date",
                categories=event_types,
                colors=[
                    "#4E79A7",
                    "#F28E2B",
                    "#E15759",
                    "#76B7B2",
                    "#59A14F",
                    "#EDC948",
                ],
                crisis_date=crisis_date,
                conflict_date=conflict_date,
            ),
            title=port.title(),
        )
    )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

#### Observations and Limitations

* To identify the impact of conflict on shipping routes, the team considered all the conflict events that occured around a 200km radius around the ports. 
* This covers roughly 73% of all the conflicts that occurred within Yemen and Jordan between 1st January 2023 and 27th February 2024. 

In [ ]:
conflict_by_country.to_csv(
    "../../data/conflict/conflict_by_country_2023-01-01_2024-02-27.csv"
)
conflict_by_port.to_csv(
    "../../data/conflict/conflict_by_port_2023-01-01_2024-02-27.csv"
)
data.to_csv("../../data/conflict/acled_raw_2023-01-01_2024-02-27.csv")